In [1]:
# Must be included at the beginning of each new notebook. Remember to change the app name.
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('1').getOrCreate()

In [2]:
# Importing data.
df = spark.read.csv('dataset/ri_statewide_2019_02_25.csv', header=True, inferSchema=True)

In [3]:
#Intial look at the data
df.show()

+--------------+----------+--------+----+------------+-----------+-------------+---------+-----------+---------------+--------------+--------+----------------+----------------+------------------+------------------+----------------+---------------+----------------+------------+-----------------+--------------------+------------+-------------+
|raw_row_number|      date|    time|zone|subject_race|subject_sex|department_id|     type|arrest_made|citation_issued|warning_issued| outcome|contraband_found|contraband_drugs|contraband_weapons|contraband_alcohol|contraband_other|frisk_performed|search_conducted|search_basis|reason_for_search|     reason_for_stop|vehicle_make|vehicle_model|
+--------------+----------+--------+----+------------+-----------+-------------+---------+-----------+---------------+--------------+--------+----------------+----------------+------------------+------------------+----------------+---------------+----------------+------------+-----------------+-----------------

In [4]:
#Print the schema
df.printSchema()

root
 |-- raw_row_number: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make

In [5]:
#Total number of values
df.count()

509681

In [6]:
df.groupBy("subject_sex").count().show()

+-----------+------+
|subject_sex| count|
+-----------+------+
|         NA| 29097|
|     female|131138|
|       male|349446|
+-----------+------+



In [7]:
df.groupBy(["zone", "subject_sex"]).count().show()

+----+-----------+-----+
|zone|subject_sex|count|
+----+-----------+-----+
|  X3|     female|26653|
|  X3|         NA| 4627|
|  X3|       male|62778|
|  X4|         NA| 9679|
|  X4|       male|94953|
|  K3|     female|29097|
|  X1|         NA| 3491|
|  X1|     female| 2702|
|  K3|       male|79771|
|  X1|       male|10522|
|  K1|         NA| 2252|
|  K2|     female|28114|
|  K1|       male|32255|
|  NA|         NA|   10|
|  K3|         NA| 4916|
|  K1|     female|13855|
|  K2|         NA| 4122|
|  X4|     female|30717|
|  K2|       male|69167|
+----+-----------+-----+



In [8]:
df.groupBy(["subject_sex", "reason_for_stop"]).count().show()

+-----------+--------------------+------+
|subject_sex|     reason_for_stop| count|
+-----------+--------------------+------+
|     female|Other Traffic Vio...| 17911|
|       male|Special Detail/Di...| 12977|
|     female|Equipment/Inspect...| 14039|
|     female|                 APB|   109|
|     female|Violation of City...|   216|
|         NA|    Call for Service|     4|
|         NA|Equipment/Inspect...|     2|
|       male|    Call for Service|  5237|
|       male|Registration Viol...| 14181|
|       male|            Speeding|182538|
|         NA|            Speeding|     8|
|       male|Motorist Assist/C...|   657|
|       male|   Suspicious Person|   268|
|     female|  Seatbelt Violation|  3550|
|     female|Registration Viol...|  5649|
|       male|Other Traffic Vio...| 72317|
|         NA|                  NA| 29073|
|     female|   Suspicious Person|    74|
|         NA|  Seatbelt Violation|     3|
|       male|                 APB|   376|
+-----------+--------------------+

In [9]:
#Drop the rows from which subject_sex is missing
#Regular dropna wasn't working cuz data is a String "NA"
df = df.filter(df.subject_sex.endswith('ale'))

In [10]:
df.groupBy("subject_sex").count().show()

+-----------+------+
|subject_sex| count|
+-----------+------+
|     female|131138|
|       male|349446|
+-----------+------+



In [11]:
df.groupBy("contraband_found").count().show()

+----------------+------+
|contraband_found| count|
+----------------+------+
|           FALSE| 11183|
|              NA|462822|
|            TRUE|  6579|
+----------------+------+



In [12]:
#total number of values in data BEFORE dropping duplicates
df.count()

480584

In [13]:
#dropping duplicates from data
df = df.dropDuplicates()

In [14]:
#total number of values in data AFTER dropping duplicates
df.count()

480584

In [15]:
#For checking what to drop
df.printSchema()

root
 |-- raw_row_number: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- department_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make

In [16]:
#Drop Prim keys from data
df = df.drop("raw_row_number", "department_id")

In [17]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- type: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)



In [18]:
#Check feature type
df.groupBy("type").count().show()

+---------+------+
|     type| count|
+---------+------+
|vehicular|480584|
+---------+------+



In [19]:
#Drop type feature from data, since it has only one value
df = df.drop('type')

In [20]:
#Checking if type was dropped
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)



In [21]:
df.groupBy("subject_race").count().show()

+--------------------+------+
|        subject_race| count|
+--------------------+------+
|               white|344716|
|               black| 68577|
|            hispanic| 53123|
|       other/unknown|  1344|
|asian/pacific isl...| 12824|
+--------------------+------+



In [22]:
#NA Values in some fields
df.groupBy("search_conducted").count().show()

+----------------+------+
|search_conducted| count|
+----------------+------+
|           FALSE|462822|
|            TRUE| 17762|
+----------------+------+



In [23]:
#NA Values in some fields
df.groupBy("contraband_found").count().show()

+----------------+------+
|contraband_found| count|
+----------------+------+
|           FALSE| 11183|
|              NA|462822|
|            TRUE|  6579|
+----------------+------+



In [24]:
from pyspark.sql.functions import when
#Create a new column for contraband_found 
df = df.withColumn("contraband_found_resolved", when(df.contraband_found == "NA", 0).otherwise(1))

In [25]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = fals

In [26]:
#Check newly created column
df.groupBy("contraband_found_resolved").count().show()

+-------------------------+------+
|contraband_found_resolved| count|
+-------------------------+------+
|                        1| 17762|
|                        0|462822|
+-------------------------+------+



In [27]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = fals

In [28]:
from pyspark.sql.functions import to_date, to_utc_timestamp


#Creating a date time 
df = df.select(to_date(df.date).alias('date'), 'zone', 'subject_race', 'subject_sex', 'arrest_made', 'citation_issued'
              , 'warning_issued', 'outcome', 'contraband_found', 'contraband_drugs', 'contraband_weapons', 'contraband_alcohol'
               , 'contraband_other', 'frisk_performed', 'search_conducted', 'search_basis', 'reason_for_search'
               , 'reason_for_stop', 'vehicle_make', 'vehicle_model', 'contraband_found_resolved')

In [29]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- outcome: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- contraband_drugs: string (nullable = true)
 |-- contraband_weapons: string (nullable = true)
 |-- contraband_alcohol: string (nullable = true)
 |-- contraband_other: boolean (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = false)



In [30]:
#Check outcome column
df.groupBy("outcome").count().show()

+--------+------+
| outcome| count|
+--------+------+
|      NA|  6763|
|citation|428378|
|  arrest| 16603|
| warning| 28840|
+--------+------+



In [31]:
df = df.drop('outcome')

In [32]:
#Drop features linearly dependent
df = df.drop('contraband_drugs', 'contraband_weapons', 'contraband_alcohol', 'contraband_other')

In [33]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- arrest_made: string (nullable = true)
 |-- citation_issued: string (nullable = true)
 |-- warning_issued: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- frisk_performed: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- search_basis: string (nullable = true)
 |-- reason_for_search: string (nullable = true)
 |-- reason_for_stop: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = false)



In [34]:
#Drop features not known prior to making the stop
df = df.drop('arrest_made', 'frisk_performed', 'search_basis', 'reason_for_search', 'reason_for_stop')

In [35]:
#Drop features not known prior to making the stop
df = df.drop('warning_issued', 'citation_issued')

In [36]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- vehicle_model: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = false)



In [37]:
#Check vehicle make column
df.groupBy("vehicle_make").count().show()

+------------+------+
|vehicle_make| count|
+------------+------+
|        MERK|     1|
|          PC|     1|
|        DODG| 14170|
|        MASE|    30|
|        PEUG|     1|
|        DATS|     7|
|        EAGL|     1|
|        STLG|    93|
|        FRUE|     9|
|        LINC|  2659|
|        INTE|     1|
|        AMER|    15|
|        MERZ|  3159|
|        TRIM|     1|
|        AMGN|    71|
|        FIAT|    44|
|        ISUZ|     1|
|          NA|162525|
|        CHRY|  7550|
|        TRIU|    18|
+------------+------+
only showing top 20 rows



In [38]:
from pyspark.sql.functions import countDistinct, avg, stddev
df.select(countDistinct("vehicle_make").alias("Distinct vehicle makes")).show()

+----------------------+
|Distinct vehicle makes|
+----------------------+
|                    97|
+----------------------+



In [39]:
#Check vehicle model column
df.groupBy("vehicle_model").count().show()

+-------------+-----+
|vehicle_model|count|
+-------------+-----+
|        ASTRO|   92|
|          MDX|  590|
|       CHR300|  123|
|          BOX|   43|
|           LT|    7|
|         545I|   30|
|      MAZDA3I|    4|
|     6 TOURIN|    2|
|     LE SABLE|    6|
|    GEO PRIZM|    5|
|          E.S|    1|
|    TL S-TYPE|    1|
|          ETK|    1|
|     TEMPO GL|    4|
|         PROS|    5|
|      LLUMINA|    3|
|    ENTOURAGE|   14|
| LEGACY WAGON|    2|
|     6 SERIES|    4|
|          ...|   15|
+-------------+-----+
only showing top 20 rows



In [40]:
from pyspark.sql.functions import countDistinct, avg, stddev
df.select(countDistinct("vehicle_model").alias("Distinct vehicle models")).show()

+-----------------------+
|Distinct vehicle models|
+-----------------------+
|                   9184|
+-----------------------+



In [41]:
#Drop vehicle model
df = df.drop('vehicle_model')

In [42]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- search_conducted: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = false)



In [43]:
#NA Values in some fields
df.groupBy("search_conducted").count().show()

+----------------+------+
|search_conducted| count|
+----------------+------+
|           FALSE|462822|
|            TRUE| 17762|
+----------------+------+



In [44]:
from pyspark.sql.functions import when
#Create a new column for contraband_found 
df = df.withColumn("search_conducted", when(df.search_conducted == "FALSE", 0).otherwise(1))

In [45]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- contraband_found: string (nullable = true)
 |-- search_conducted: integer (nullable = false)
 |-- vehicle_make: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = false)



In [46]:
from pyspark.sql.functions import when
#Create a new column for contraband_found 
df = df.withColumn("contraband_found", when(df.contraband_found == "FALSE", 0).otherwise(1))

In [47]:
df.printSchema()

root
 |-- date: date (nullable = true)
 |-- zone: string (nullable = true)
 |-- subject_race: string (nullable = true)
 |-- subject_sex: string (nullable = true)
 |-- contraband_found: integer (nullable = false)
 |-- search_conducted: integer (nullable = false)
 |-- vehicle_make: string (nullable = true)
 |-- contraband_found_resolved: integer (nullable = false)



In [48]:
#Imports for assembler, encoder, indexer
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [49]:
#Working with categorical data

# First create a string indexer (convert every string into a number, such as male = 0 and female = 1).
# A number will be assigned to every category in the column.
gender_indexer = StringIndexer(inputCol='subject_sex',outputCol='SexIndex')

# Now we can one hot encode these numbers. This converts the various outputs into a single vector.
# This makes it easier to process when you have multiple classes.
gender_encoder = OneHotEncoder(inputCol='SexIndex',outputCol='SexVec')

#Similar to the above.
race_indexer = StringIndexer(inputCol='subject_race',outputCol='raceIndex')
race_encoder = OneHotEncoder(inputCol='raceIndex',outputCol='raceVec')

#Similar to the above.
zone_indexer = StringIndexer(inputCol='zone',outputCol='zoneIndex')
zone_encoder = OneHotEncoder(inputCol='zoneIndex',outputCol='zoneVec')

#Similar to the above.
vehicle_indexer = StringIndexer(inputCol='vehicle_make',outputCol='vehicleIndex')
vehicle_encoder = OneHotEncoder(inputCol='vehicleIndex',outputCol='vehicleVec')

In [58]:
# Now we can assemble all of this as one vector in the features column. 
assembler = VectorAssembler(inputCols=['date', 
 'SexVec',
 'zoneVec',
 'search_conducted',
 'vehicleVec',                                      
 'raceVec'],outputCol='features')

In [59]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [60]:
# Note that survived is a categorial variable but didn't require any transformation.
# That's because it's already in the format of 1's and 0's. 
log_reg = LogisticRegression(featuresCol='features',labelCol='contraband_found')

In [61]:
# Lists everything we want to do. Index data, encode data, assemble data and then pass in the actual model.
pipeline = Pipeline(stages=[gender_indexer,race_indexer, zone_indexer, vehicle_indexer, 
                           gender_encoder,race_encoder, zone_encoder, vehicle_encoder,
                           assembler,log_reg])

In [62]:
# Train/test split. 
train_data, test_data = df.randomSplit([0.7,.3])

In [63]:
# Note pipeline. Call it as you would call a machine learning object.
fit_model = pipeline.fit(train_data)

IllegalArgumentException: 'Data type DateType is not supported.'

In [56]:
# Transform test data. 
results = fit_model.transform(test_data)

In [57]:
#We can then evaluate using AUC (area under the curve). AUC is linked to ROC.
AUC = my_eval.evaluate(results)

AUC

NameError: name 'my_eval' is not defined